# Pattern recognition CW2

In [1]:
from scipy.io import loadmat
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import time

### Import file with 6 main components:

In [4]:
""" 
camId : which camera was used to get the shot (1 or 2)
filelist: names of the images (with format x_label_camId_index.png)
labels: class of the image (which person's image is it?)
query_idx: indexes of test set
gallery_idx: indexes of test set used for kNN 
train_idx: indexes of training and validation set
"""   
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')

### Import file with the feature vectors of all images:

In [5]:
import json
with open('PR_data/feature_data.json', 'r') as f:
    features = json.load(f)
features_np = np.array(features) #list of features converted to an array 
features_np.shape

(14096, 2048)

In [405]:
np.unique(train_idxs['labels'])[-1]

1467

In [240]:
train_idxs['gallery_idx'].shape

array([[   21],
       [   23],
       [   24],
       ...,
       [14062],
       [14064],
       [14065]], dtype=uint16)

In [242]:
train_idxs['train_idx']

array([[    1],
       [    2],
       [    3],
       ...,
       [14094],
       [14095],
       [14096]], dtype=uint16)

In [9]:
7368+5328

12696

In [10]:
train_idxs['query_idx'].shape

(1400, 1)

In [11]:
12696+1400

14096

## kNN Classification for query set on gallery set

In [108]:
def extract_info_from_filename(image_index): 
    name = str(train_idxs['filelist'][image_index][0][0])
    [dc, label, camId, index] = name.split('_')
    return label, camId

def delete_images(query_image, query_index, gallery_images, gallery_labels):
    gallery_names = train_idxs['filelist'][train_idxs['gallery_idx'].flatten()]
    query_label, query_camId = extract_info_from_filename(query_index)
    new_gallery_images = []
    new_gallery_labels = []
    for img, name in zip(gallery_images, gallery_names):
        [dc, label_g, camId_g, index] = str(name[0][0]).split('_')
        if [int(label_g), int(camId_g)] != [int(query_label), int(query_camId)]:
            new_gallery_images.append(img)
            new_gallery_labels.append(int(label_g))
    return new_gallery_images, new_gallery_labels

def score_rank(query_label, rank_k, new_gallery_labels):
    #print(rank_k)
    rank_k_labels = np.array(new_gallery_labels)[rank_k #extract the labels which are of rank k
    return (query_label[0] in rank_k_labels) #return true if query label in rank k, else false

In [229]:
train_idxs['gallery_idx'].flatten()

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [109]:
def NNClassification_deletions(query_image, query_index, gallery_images, gallery_labels, k):
    neigh = NearestNeighbors(n_neighbors=k, n_jobs=-1)
    new_gallery_images, new_gallery_labels = delete_images(query_image, query_index, gallery_images, gallery_labels)
    neigh.fit(new_gallery_images)   #new_gallery_labels)
    distances, indices = neigh.kneighbors(query_image, k)
    return distances, indices, new_gallery_labels #neigh.score(query_image, query_label), neigh.predict(query_image), 

In [110]:
gallery_labels = train_idxs['labels'][train_idxs['gallery_idx'].flatten()]
query_labels = train_idxs['labels'][train_idxs['query_idx'].flatten()]

query_images = features_np[train_idxs['query_idx'].flatten()]
gallery_images = features_np[train_idxs['gallery_idx'].flatten()]

def get_accuracy_for_k_ranks(k):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()):
        if(index < 101):
            if (index % 10 == 0):
                print("Index: ", index, " & time taken: ", time.time() - start)

            #perform NN Classification and extract top k ranks 
            dist, rank_k_indices, new_gallery_labels = NNClassification_deletions(query_images[index].reshape(1, 2048), query_index, gallery_images, gallery_labels, k)
            #calculate the score (true/false) for top k images and query image recognition
            score = score_rank(query_labels[index], rank_k_indices, new_gallery_labels)
            #create a list of scores to get overall accuracy later on
            list_of_truths.append(score)
    
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)

In [111]:
train_idxs['gallery_idx'].flatten()

array([   21,    23,    24, ..., 14062, 14064, 14065], dtype=uint16)

In [235]:
train_idxs['filelist'][204]

array([array(['1_022_2_06.png'], dtype='<U14')], dtype=object)

In [228]:
query_images

array([[0.98361254, 0.33195475, 0.26549727, ..., 0.02350602, 0.08491972,
        0.00514889],
       [1.30696166, 0.22219124, 0.62375975, ..., 0.02954952, 0.37348923,
        0.05303315],
       [0.29322624, 0.55588913, 0.06961903, ..., 0.15892459, 0.21527511,
        0.09551907],
       ...,
       [0.7593497 , 0.24537075, 2.19513273, ..., 0.47061139, 1.35971546,
        0.46198806],
       [0.48667279, 0.49618778, 0.30824399, ..., 0.04450084, 1.28799629,
        0.37683338],
       [0.05719076, 0.60635394, 0.58300596, ..., 0.40994745, 1.37061644,
        0.55975795]])

In [400]:
gallery_labels = train_idxs['labels'][train_idxs['gallery_idx'].flatten()]
query_labels = train_idxs['labels'][train_idxs['query_idx'].flatten()]

query_images = features_np[train_idxs['query_idx'].flatten()]
gallery_images = features_np[train_idxs['gallery_idx'].flatten()]

N=100


def remove_indices(n_indices, query_index):
    query_label, query_camId = extract_info_from_filename(query_index)
    n_names = train_idxs['filelist'][n_indices]
    #k_images = features_np[k_indices]
    #print(k_names.shape, k_names[0], k_images[0])
    final_n_indices = []
    final_n_labels = []
    for index, name in zip(n_indices, n_names):
        [dc, label_g, camId_g, index_g] = str(name[0][0]).split('_')
        #print(str(name[0][0]).split('_'))
        #print(index)
        if [int(label_g), int(camId_g)] != [int(query_label), int(query_camId)]:
            final_n_indices.append(index)
            final_n_labels.append(int(label_g))
    return final_n_indices, final_n_labels

def get_accuracy(k, N, gallery_images):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1, metric='euclidean')
    neigh.fit(gallery_images)   #new_gallery_labels)
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()):
        if (index % 10 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh.kneighbors(query_images[index].reshape(1, 2048), N)
        #print(query_labels[index], query_index, N_indices)
        topN_gallery_images = gallery_images[N_indices[0]]
        forLoopStart = time.time()
        topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
        #topN_gallery_indices = [np.where(np.all(features_np == image, axis=1))[0][0] for image in topN_gallery_images]
        #print(time.time()-forLoopStart)
        #print('topN indices', topN_gallery_indices.flatten().tolist())
        #print('n_indices', N_indices[0])
        #print(train_idxs['gallery_idx'][N_indices[0]])
        reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
        if index == 300:
            break
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    print(acc)

In [401]:
features_np[[2, 3]].shape

(2, 2048)

In [402]:
%prun get_accuracy(1, 100)

Index:  0  & time taken:  1.1097545623779297
Index:  10  & time taken:  2.233020067214966
Index:  20  & time taken:  3.304171323776245
Index:  30  & time taken:  4.40855860710144
Index:  40  & time taken:  5.501349449157715
Index:  50  & time taken:  6.595091819763184
Index:  60  & time taken:  7.727085828781128
Index:  70  & time taken:  8.817773818969727
Index:  80  & time taken:  9.915739059448242
Index:  90  & time taken:  11.015377283096313
Index:  100  & time taken:  12.136348009109497
Index:  110  & time taken:  13.257834434509277
Index:  120  & time taken:  14.34682846069336
Index:  130  & time taken:  15.468301057815552
Index:  140  & time taken:  16.57993507385254
Index:  150  & time taken:  17.694861888885498
Index:  160  & time taken:  18.808980703353882
Index:  170  & time taken:  19.907105445861816
Index:  180  & time taken:  20.996891498565674
Index:  190  & time taken:  22.087839126586914
Index:  200  & time taken:  23.182602882385254
Index:  210  & time taken:  24.2870

## K-means

In [ ]:
from sklearn.cluster import KMeans

total_number_of_labels = np.unique(train_idxs['labels'])[-1]
n_clusters = total_number_of_labels #total_number_of_labels

kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(gallery_images)
#print(kmeans.labels_)
index_and_cluster = [(index, label) for index, label in zip(train_idxs['gallery_idx'].flatten(), kmeans.labels_)]
#np.unique(kmeans.labels_)

In [461]:
def img_indices_in_cluster(c): 
    cluster_labels = [pair[0] for pair in index_and_cluster if pair[1] == c]
    return cluster_labels

In [462]:
def img_cluster(k, N, c_g_indices, query_index, index):
    list_of_truths = []
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1, metric='euclidean')
    neigh.fit(features_np[c_g_indices])   #new_gallery_labels)
    
    
    N_distances, N_indices = neigh.kneighbors(features_np[query_index].reshape(1, -1), N)
    #print(query_labels[index], query_index, N_indices)
    #topN_c_g_images = features_np[c_g_indices][N_indices[0]]
    topN_c_g_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
    
    reduced_topN_indices, reduced_topN_labels = remove_indices(topN_c_g_indices, query_index)
    
    return (query_labels[index][0] in reduced_topN_labels[:k])

def knn_clustering(k, N):
    class_success = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()):
        if (index % 10 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        cluster = kmeans.predict(query_images[index].reshape(1,-1))
        cluster_gallery_indices = img_indices_in_cluster(cluster)
        class_success.append(img_cluster(k, N, cluster_gallery_indices, query_index, index))
        
        
        if index == 100:
            break
     #over-all accuracy
    acc = sum(class_success)/len(class_success)
    print(acc)

In [ ]:
k = 10
N = 100
knn_clustering(k, N)

In [412]:
#kmeans.predict(query_images[1])
for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()):
    cluster = kmeans.predict(query_images[index].reshape(1,-1))
    
    cluster_gallery_indices = img_indices_in_cluster(cluster)
    
    
    #actual_label = query_labels[index]
    #if predicted_label == actual_label:
        


#KMeansClustering(total_number_of_labels)

ValueError: Expected 2D array, got 1D array instead:
array=[1.30696166 0.22219124 0.62375975 ... 0.02954952 0.37348923 0.05303315].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [415]:
kmeans.predict(query_images[1].reshape(1, -1))

array([797])

In [418]:
query_labels[1]

array([3], dtype=uint16)